In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yfinance as yf
from machine_learning_finance import analyze_trades, \
    calc_probabilties_without_lookahead, plot_backtest_analysis, \
    metrics_to_dataframe, create_train_test_windows
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "XLE"
file = f"../backtests/backtest_inverse_{symbol}_365_h0.9_l0.1.csv"
period = 365
pd.options.display.max_rows = None


def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()

def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)
    test_df = calc_probabilties_without_lookahead(test_df, hist_df)
    print("oh boy:", test_df.index.min())
    print("oh boy:", test_df.index.max())
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    hist_df = df.head(len(df)-period)
    hist_df = hist_df.tail(period*4)
    df = df.tail(period)
    df = calc_probabilties_without_lookahead(df, hist_df)
    plot_backtest_analysis(df, ledger, inverse=symbol)

    return ledger, df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)
#plot_win_loss(file, metrics)

#print(ledger)

#print(metrics_df)
#df.sort_index(ascending=False)

[*********************100%***********************]  1 of 1 completed


In [12]:
#ledger, df = analyze_and_graph(file, symbol, 365*4, df)    
ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 

merged_df = ledger.merge(df2, left_on='Date', right_index=True, how='left')
merged_df[["Date", "Price", "Product", "Side", "Action","prob_above_trend"]]

wtf test min: 2020-03-27 00:00:00
wtf test max: 2021-12-17 00:00:00
oh boy: 2020-03-27 00:00:00
oh boy: 2021-12-17 00:00:00


,Date,Price,Product,Side,Action,prob_above_trend
0,2020-03-27,28.613300,XLE,long,enter,0.999992
1,2020-06-03,42.147300,XLE,long,exit,0.083570
2,2020-06-03,193.263506,DUG,long,enter,0.083570
3,2020-07-09,245.833994,DUG,long,exit,0.930726
4,2020-07-09,34.784399,XLE,long,enter,0.930726
5,2020-08-05,37.804300,XLE,long,exit,0.025179
6,2020-08-05,202.353506,DUG,long,enter,0.025179
7,2020-09-23,288.456006,DUG,long,exit,0.957116
8,2020-09-23,30.512099,XLE,long,enter,0.957116
9,2020-11-24,40.713101,XLE,long,exit,0.018688


In [8]:
df.tail(10)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-05-05,80.269997,80.860001,79.790001,80.230003,80.230003,19533600
2023-05-08,81.400002,81.750000,80.220001,80.230003,80.230003,16355900
2023-05-09,79.599998,81.019997,79.430000,80.260002,80.260002,20710600
2023-05-10,80.660004,80.820000,78.750000,79.349998,79.349998,18026000
2023-05-11,78.230003,78.809998,77.800003,78.400002,78.400002,19084100
2023-05-12,78.790001,79.320000,77.949997,78.519997,78.519997,18952700
2023-05-15,78.809998,79.370003,78.190002,78.919998,78.919998,16272300
2023-05-16,78.550003,78.889999,76.860001,76.980003,76.980003,22255600
2023-05-17,77.769997,78.879997,77.199997,78.459999,78.459999,23100300
